In [14]:
df = spark.read.format("csv").option("header","true").load("Files/Products/products.csv")
# df now is a Spark DataFrame containing CSV data from "abfss://DP_FABRIC@onelake.dfs.fabric.microsoft.com/DPFABRIC.Lakehouse/Files/Products/products.csv".
display(df)

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, e884b25a-9db7-4d4d-9e60-54d5456824cb)

In [15]:
df.write.format("delta").saveAsTable("managed_products")

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 18, Finished, Available)

In [16]:
df.write.format("delta").saveAsTable("external_products", path="abfss://DP_FABRIC@onelake.dfs.fabric.microsoft.com/DPFABRIC.Lakehouse/Files/external_products")

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 19, Finished, Available)

In [17]:
%%sql

DESCRIBE FORMATTED managed_products;

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 20, Finished, Available)

<Spark SQL result set with 12 rows and 3 fields>

In [18]:
%%sql

DESCRIBE FORMATTED external_products;

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 21, Finished, Available)

<Spark SQL result set with 12 rows and 3 fields>

In [19]:
%%sql

DROP TABLE managed_products;
DROP TABLE external_products;

StatementMeta(, , -1, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

<Spark SQL result set with 0 rows and 0 fields>

In [20]:
%%sql

CREATE TABLE products
USING DELTA
LOCATION 'Files/external_products';

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 24, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [21]:
%%sql

SELECT * FROM products;

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 25, Finished, Available)

<Spark SQL result set with 295 rows and 4 fields>

In [22]:
%%sql

UPDATE products
SET ListPrice = ListPrice * 0.9
WHERE Category = 'Mountain Bikes';

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 26, Finished, Available)

<Spark SQL result set with 1 rows and 1 fields>

In [23]:
%%sql

DESCRIBE HISTORY products;

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 27, Finished, Available)

<Spark SQL result set with 2 rows and 15 fields>

In [24]:
delta_table_path = 'Files/external_products'

# Get the current data
current_data = spark.read.format("delta").load(delta_table_path)
display(current_data)

# Get the version 0 data
original_data = spark.read.format("delta").option("versionAsOf", 0).load(delta_table_path)
display(original_data)

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 28, Finished, Available)

SynapseWidget(Synapse.DataFrame, 3efe67cb-534d-440f-9bd4-888a9a4f5ea1)

SynapseWidget(Synapse.DataFrame, 9f8f4dc6-0254-4109-841c-da49f8939831)

In [25]:
from notebookutils import mssparkutils
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Create a folder
inputPath = 'Files/data/'
mssparkutils.fs.mkdirs(inputPath)

# Create a stream that reads data from the folder, using a JSON schema
jsonSchema = StructType([
StructField("device", StringType(), False),
StructField("status", StringType(), False)
])
iotstream = spark.readStream.schema(jsonSchema).option("maxFilesPerTrigger", 1).json(inputPath)

# Write some event data to the folder
device_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"ok"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''
mssparkutils.fs.put(inputPath + "data.txt", device_data, True)
print("Source stream created...")

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 29, Finished, Available)

Source stream created...


In [26]:
# Write the stream to a delta table
delta_stream_table_path = 'Tables/iotdevicedata'
checkpointpath = 'Files/delta/checkpoint'
deltastream = iotstream.writeStream.format("delta").option("checkpointLocation", checkpointpath).start(delta_stream_table_path)
print("Streaming to delta sink...")

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 30, Finished, Available)

Streaming to delta sink...


In [27]:
%%sql

SELECT * FROM IotDeviceData;

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 31, Finished, Available)

<Spark SQL result set with 9 rows and 2 fields>

In [28]:
# Add more data to the source stream
more_data = '''{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"ok"}
{"device":"Dev1","status":"error"}
{"device":"Dev2","status":"error"}
{"device":"Dev1","status":"ok"}'''

mssparkutils.fs.put(inputPath + "more-data.txt", more_data, True)

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 32, Finished, Available)

True

In [29]:
%%sql

SELECT * FROM IotDeviceData;

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 33, Finished, Available)

<Spark SQL result set with 16 rows and 2 fields>

In [30]:
deltastream.stop()

StatementMeta(, 37e9a0e5-d98c-44ed-8c79-650a1c770c39, 34, Finished, Available)